In [1]:
# !pip install ultralytics

In [2]:
import os
from ultralytics import YOLO
import numpy as np
import cv2
import pandas as pd


In [61]:
folder = "boss_images"
# folder = "zhizha"

In [62]:
os.listdir(folder)

['ac723806c7124629afdf8ec1009867f0_hrs.jpg',
 'FREIGHTLINER_CENTURY_CLASS_00050.jpg',
 'FREIGHTLINER_CENTURY_CLASS_00054.jpg']

In [63]:

class YOLOSegmentation:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def detect(self, img):
        height, width, channels = img.shape

        results = self.model.predict(source=img.copy(), save=False, save_txt=False)
        result = results[0]
        segmentation_contours_idx = []
        for seg in result.masks.xyn:
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)

        bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)
        return bboxes, class_ids, segmentation_contours_idx, scores


In [87]:
def extract_contours(folder: str, Segmentator: YOLOSegmentation) -> None:
    for image_name in os.listdir(folder):
        if not (image_name.endswith(".jpg") or image_name.endswith(".jpeg")):
            continue
        start_path = folder + '/' + image_name
        end_path = f"{folder}/processed"
        os.makedirs(end_path, exist_ok=True)

        img = cv2.imread(start_path)
        if img is None:
            print(Exception(f"Can't read an image: {image_name}"))
            continue
        img = cv2.resize(img, None, fx=0.5, fy=0.5)

        bboxes, classes, segmentations, scores = Segmentator.detect(img)


        # for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):
        #     (x, y, x2, y2) = bbox
        #
        #     cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 2)
        #     cv2.polylines(img, [seg], True, (0, 0, 255), 4)
        #     edges = cv2.putText(img, str(class_id), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        #
        #     image_name_pieces = image_name.split(".")
        #     image_name = image_name_pieces[0] + "_processed" + "." + image_name_pieces[-1]
        #     cv2.imwrite(end_path + "/" + image_name, edges)
        # continue



        points = np.array(segmentations[0])

        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)

        img = cv2.bitwise_and(img, img, mask=mask)


        blurred_image = cv2.GaussianBlur(img.copy(),(5,5),0)

        edges = cv2.Canny(blurred_image, 100, 160)

        image_name_pieces = image_name.split(".")
        image_name = image_name_pieces[0] + "_processed." + image_name_pieces[-1]
        # print(image_name, folder + '/' + image_name)
        cv2.imwrite(end_path + "/" + image_name, edges)


In [88]:
extract_contours(folder=folder, Segmentator=YOLOSegmentation("yolov8m-seg.pt"))


0: 448x640 5 persons, 2 cars, 3 trucks, 1443.0ms
Speed: 7.0ms preprocess, 1443.0ms inference, 27.0ms postprocess per image at shape (1, 3, 448, 640)

0: 384x640 3 trucks, 1173.0ms
Speed: 3.0ms preprocess, 1173.0ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 car, 3 trucks, 2005.4ms
Speed: 3.0ms preprocess, 2005.4ms inference, 15.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 1 truck, 1728.0ms
Speed: 6.0ms preprocess, 1728.0ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 3 trucks, 1517.0ms
Speed: 3.0ms preprocess, 1517.0ms inference, 14.0ms postprocess per image at shape (1, 3, 480, 640)

0: 512x640 2 cars, 1 truck, 1529.0ms
Speed: 4.0ms preprocess, 1529.0ms inference, 14.0ms postprocess per image at shape (1, 3, 512, 640)

0: 384x640 2 cars, 1130.0ms
Speed: 4.0ms preprocess, 1130.0ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 2 cars, 2091.0ms
Speed: 4.0ms